In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from http.cookies import SimpleCookie
import requests
import os
import time

In [13]:
# 다운로드 경로 설정
current_dir = os.getcwd()
download_path = current_dir + '\\Downloads\\'

chrome_options = uc.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
  "profile.default_content_setting_values.popups": 1,  # 0: 모든 팝업 차단, 1: 모든 팝업 허용
  "profile.default_content_setting_values.notifications": 1,  # 1: 허용, 2: 차단
  "download.default_directory": download_path,  # 다운로드 경로
  "download.prompt_for_download": False,  # 다운로드시 확인 창 띄우지 않음
  "download.directory_upgrade": True,  # 다운로드 경로 설정 활성화
  "safebrowsing.enabled": True,  # 안전 브라우징 활성화
  "profile.default_content_setting_values.automatic_downloads": 1
})

# 드라이버 설정
driver = uc.Chrome(options=chrome_options)
driver.get('https://www.google.com/')

In [14]:
# 구글북스 예제
driver.get('https://play.google.com/books/reader?id=6ybeEAAAQBAJ&hl=ko')
cookies = driver.get_cookies()
print(cookies)

[{'domain': '.play.google.com', 'expiry': 1750349445, 'httpOnly': False, 'name': '_ga_9JS7FWYMLB', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1715789445.1.0.1715789445.0.0.0'}, {'domain': '.google.com', 'expiry': 1747325444, 'httpOnly': False, 'name': 'SIDCC', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'AKEyXzUofw1JAP9uzJg97OXcyfCj9r1GthBzkVqupQ5OAJOZAo9KtDPidLSLopcvM9U_F00-'}, {'domain': '.google.com', 'expiry': 1747325444, 'httpOnly': True, 'name': '__Secure-1PSIDCC', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'AKEyXzVJfxagLp_PQYTIpvS5aoxOryLBJiKMZCat_SfDBJyRiNeM3Jc7_MGZwQ16r9LApFgoFQ'}, {'domain': '.play.google.com', 'httpOnly': False, 'name': 'PLAY_ACTIVE_ACCOUNT', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'ICrt_XL61NBE_S0rhk8RpG0k65e0XwQVdDlvB6kxiQ8=authuser-0'}, {'domain': '.google.com', 'expiry': 1747325443, 'httpOnly': True, 'name': '__Secure-3PSIDTS', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'si

#### 구글 로그인 이후 작업

In [34]:
book_list = []
force_startnum = -1

In [16]:
# 새 탭 열기 테스트 (브라우저의 팝업 허용 메시지가 뜰 수 있음.)

def open_new_tab_and_close(driver, url):
    # 현재 탭의 핸들 저장
    original_tab = driver.current_window_handle

    # 새 탭 열기
    driver.execute_script("window.open('about:blank');")
    new_tab = driver.window_handles[-1]
    driver.switch_to.window(new_tab)

    # 새 탭에서 원하는 URL로 이동
    driver.get(url)
    time.sleep(3)
    # 여기에 필요한 작업을 추가
    # 예: 데이터 수집, 다운로드 등

    # 새 탭 닫기 및 원래 탭으로 돌아가기
    driver.switch_to.window(original_tab)

#open_new_tab_and_close(driver, "https://google.com/")

In [24]:
def func(list):
    driver.switch_to.window(driver.window_handles[0])
    cookies = driver.get_cookies()
    
    # iframe 대기 및 포커스 이동
    iframe = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe.-gb-display"))
    )
    driver.switch_to.frame(iframe)

    # 필요한 요소 대기 및 찾기
    wrapper = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.CLASS_NAME, "cdk-virtual-scroll-content-wrapper"))
    )

    ol_element = wrapper.find_element(By.TAG_NAME, "ol")
    li_elements = ol_element.find_elements(By.TAG_NAME, "li")

    for li in li_elements:
        render = li.find_element(By.TAG_NAME, "reader-rendered-page")
        img = render.find_element(By.TAG_NAME, "img")
        src_value = img.get_attribute('src')
        #src_value = img.get_attribute('real-src')
        print(src_value)
        if src_value not in list:
            list.append(src_value)
            last_index = len(list) - 1 + force_startnum
            #download_requests(src_value, last_index, cookies)
            download_selenium(src_value, last_index)
    
    return wrapper

# iframe 내에서 스크롤
def scroll(wrapper):
    ol_element = wrapper.find_element(By.TAG_NAME, "ol")
    li_elements = ol_element.find_elements(By.TAG_NAME, "li")
    driver.execute_script("arguments[0].scrollIntoView();", li_elements[-1])

def download_requests(image_url, file_name, cookies):
    # 쿠키를 가져와서 requests 세션에 적용
    session = requests.Session()
        
    for cookie in cookies:
        cookie_obj = requests.cookies.create_cookie(
            domain=cookie.get('domain', ''),
            path=cookie.get('path', '/'),
            name=cookie['name'],
            value=cookie['value'],
            secure=cookie.get('secure'),
            rest={'HttpOnly': cookie.get('httpOnly', False)},
            expires=cookie.get('expiry')
        )
        session.cookies.set_cookie(cookie_obj)

    cookie_string = '; '.join([f"{cookie['name']}={cookie['value']}" for cookie in cookies])

    print("쿠키는:", cookie_string)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        #'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'Referer': 'https://play.google.com/books/reader?id=6ybeEAAAQBAJ&pg=GBS.PA107&hl=ko',  # 이 부분은 실제 참조하는 페이지 URL로 변경해야 합니다.
        'Cookie': cookie_string
    }

    response = requests.get(image_url, headers=headers)
    if response.status_code == 200:
        # 이미지 저장 경로와 파일 이름 설정
        image_path = os.path.join('downloads', f'{file_name}.jpg')
        print(f'{file_name}.jpg')
        os.makedirs(os.path.dirname(image_path), exist_ok=True)
        
        # 이미지 파일로 저장
        with open(image_path, 'wb') as file:
            file.write(response.content)
    
def download_selenium(image_url, file_name):
    # https://blog.naver.com/PostView.naver?blogId=tjswo0077&logNo=222084415732
    javascript = """(function() {
    // blob URL
    var blobUrl = \"""" + image_url + """\";

    // blob URL을 a 태그로 다운로드
    function downloadBlob(blobUrl, fileName) {
        // Fetch the blob object from the blob URL
        fetch(blobUrl)
            .then(response => response.blob())
            .then(blob => {
                // Create a temporary URL for the blob
                var url = window.URL.createObjectURL(blob);
                var a = document.createElement('a');
                a.style.display = 'none';
                a.href = url;
                a.download = fileName;
                document.body.appendChild(a);
                a.click();
                window.URL.revokeObjectURL(url);
                document.body.removeChild(a);
            })
            .catch(error => console.error('이미지 다운로드 실패:', error));
    }

    // Download the blob with the given file name
    downloadBlob(blobUrl, '""" + str(file_name) + """.png');
})();"""
    driver.execute_script(javascript)
    

In [35]:
while(True):
    wrapper = func(book_list)
    scroll(wrapper)
    time.sleep(1.2)

blob:https://books.googleusercontent.com/9ceabe60-8ae5-4c41-8695-b03f6aea2c72
blob:https://books.googleusercontent.com/d629dd93-d487-438d-b236-a03ba9d36147
blob:https://books.googleusercontent.com/ec945ca6-b64e-4958-8f88-371846cd783a
blob:https://books.googleusercontent.com/0d6d53f0-c4b0-4308-86b9-997d7654b11d
blob:https://books.googleusercontent.com/7a474f6e-89a1-4823-9720-43a27e954607
blob:https://books.googleusercontent.com/002d70dd-6f55-4258-90d0-4a6b7d19ed0d
blob:https://books.googleusercontent.com/74493aee-9e76-4362-bc5b-3ddcbd851cab
blob:https://books.googleusercontent.com/f7f32187-07ff-4958-855a-80319a04c3c6
blob:https://books.googleusercontent.com/901d96c1-cbbc-4f8e-bc7e-cb25f41f68b0
blob:https://books.googleusercontent.com/002d70dd-6f55-4258-90d0-4a6b7d19ed0d
blob:https://books.googleusercontent.com/74493aee-9e76-4362-bc5b-3ddcbd851cab
blob:https://books.googleusercontent.com/f7f32187-07ff-4958-855a-80319a04c3c6
blob:https://books.googleusercontent.com/901d96c1-cbbc-4f8e-bc7e

KeyboardInterrupt: 

In [10]:
wrapper = func(book_list)
scroll(wrapper)

blob:https://books.googleusercontent.com/e3df370e-c700-4c59-b4aa-0bd41b2bc66c
blob:https://books.googleusercontent.com/f36ca42c-365f-4a92-a33a-e3c8be55d2e8
blob:https://books.googleusercontent.com/f96caa93-7211-40c7-8162-3d7369ca2295
blob:https://books.googleusercontent.com/8248f849-9e8c-4083-a842-c311ae8e87d2
blob:https://books.googleusercontent.com/1ef73117-ca50-443f-b078-7554985d5e73
blob:https://books.googleusercontent.com/828993cc-c0c2-4cb2-b536-12b4b9cf4df5


In [23]:
image_url = "https://play.google.com/books/publisher/content/reader?id=6ybeEAAAQBAJ&hl=ko&pg=PA208&img=1&zoom=3&subver=ACfU3U2mbEHKdUZ9rszgUXD7Z-kYeERgBA&sig=ACfU3U0BBy0wBHPmiJ1NVwnE9T4FFKso6w&w=640&h=826&edge=stretch"
file_name = 1


current_dir = os.getcwd()
path = current_dir + '\\Images\\' + str(file_name) + '.png'
javascript = """(function() {
    // blob URL
    var blobUrl = \"""" + image_url + """\";

    // blob URL을 a 태그로 다운로드
    function downloadBlob(blobUrl, fileName) {
        // Fetch the blob object from the blob URL
        fetch(blobUrl)
            .then(response => response.blob())
            .then(blob => {
                // Create a temporary URL for the blob
                var url = window.URL.createObjectURL(blob);
                var a = document.createElement('a');
                a.style.display = 'none';
                a.href = url;
                a.download = fileName;
                document.body.appendChild(a);
                a.click();
                window.URL.revokeObjectURL(url);
                document.body.removeChild(a);
            })
            .catch(error => console.error('이미지 다운로드 실패:', error));
    }

    // Download the blob with the given file name
    downloadBlob(blobUrl, '""" + str(file_name) + """.png');
})();"""

print(javascript)

(function() {
    // blob URL
    var blobUrl = "https://play.google.com/books/publisher/content/reader?id=6ybeEAAAQBAJ&hl=ko&pg=PA208&img=1&zoom=3&subver=ACfU3U2mbEHKdUZ9rszgUXD7Z-kYeERgBA&sig=ACfU3U0BBy0wBHPmiJ1NVwnE9T4FFKso6w&w=640&h=826&edge=stretch";

    // blob URL을 a 태그로 다운로드
    function downloadBlob(blobUrl, fileName) {
        // Fetch the blob object from the blob URL
        fetch(blobUrl)
            .then(response => response.blob())
            .then(blob => {
                // Create a temporary URL for the blob
                var url = window.URL.createObjectURL(blob);
                var a = document.createElement('a');
                a.style.display = 'none';
                a.href = url;
                a.download = fileName;
                document.body.appendChild(a);
                a.click();
                window.URL.revokeObjectURL(url);
                document.body.removeChild(a);
            })
            .catch(error => console.error('이미지 다운로드 실패:',